In [2]:
import numpy as np
import pandas as pd
import os
import time
from datetime import date
import re
from selenium import webdriver
# from BeautifulSoup4 import BeautifulSoup
from bs4 import BeautifulSoup
import requests
from playwright.sync_api import sync_playwright#,async_playwright
from playwright.async_api import async_playwright
import asyncio
from scipy.stats import poisson

In [3]:
def calculateProb(expg1,expg2):
    # Expected goals by the the two oppsing teams.
#     expg1 = 1.1
#     expg2 = 1.9
    # The upper limit of how many goals to compute probabilities for.
    maxgoal = 15

    ## Lists for the individual probabilities
    pb1 = []
    pb2 = []

    for i in np.arange(maxgoal):
        pb1.append(poisson.pmf(i, expg1))
        pb2.append(poisson.pmf(i, expg2))

    # Assuming the independent Poisson model.
    probmat = np.matmul(np.array(pb1).reshape((15,1)),
                        np.transpose(np.array(pb2).reshape((15,1))))

    # Compute the BTTS probability using the formula from the paper.
    prob_btts = 1 - (sum(probmat[1:probmat.shape[0],0]) + \
                     sum(probmat[0,1:probmat.shape[1]]) + \
                     probmat[0,0])
    
    return prob_btts.round(4)

In [4]:
## Extract data
# %run -i testFile.py

In [5]:
os.listdir('data/2022-02-20')

['bundesliga.xlsx',
 'championship.xlsx',
 'laliga.xlsx',
 'ligue-1.xlsx',
 'premier-league.xlsx',
 'serie-a.xlsx',
 'superliga.xlsx']

In [22]:

if os.path.isdir('data/'+str(date.today())):
    # Folder with data
    dataFolder = os.listdir('data/'+str(date.today()))
    print('Folder already in place.\n\Loading the data...')
    
    # Dictonary to hold data
    dfs = {}
    
    # Read the leagues one by one
    for i,league in enumerate(dataFolder):
        temp = pd.read_excel('data/'+str(date.today())+'/'+league,
                                                  encoding='latin-1',
                                                  usecols=['Timestamp','Home team','Away team','Goal (home)','Goal (away)'])
        temp.loc[:,'Date'] = temp.Timestamp.apply(lambda x: pd.Timestamp(2022,
                                                        int(x.split(' ')[0].split('.')[1]),
                                                        int(x.split(' ')[0].split('.')[0]),
                                                        int(x.split(' ')[1].split(':')[0]),
                                                        int(x.split(' ')[1].split(':')[1])
                                                       )
                                 if int(x.split(' ')[0].split('.')[1]) < 6 else
                                 pd.Timestamp(2021,
                                              int(x.split(' ')[0].split('.')[1]),
                                              int(x.split(' ')[0].split('.')[0]),
                                              int(x.split(' ')[1].split(':')[0]),
                                              int(x.split(' ')[1].split(':')[1])
                                                       )
                                )
        dfs[league.split('.')[0]] = temp
#         df = pd.read_excel('data/'+str(date.today())+'/'+league,
#                             encoding='latin-1',
#                            usecols=['Timestamp','Home team','Away team','Goal (home)','Goal (away)'])
#     df = 
else:
    
    print('No data for today - run the script.')
#     os.mkdir('data/'+str(date.today()))

Folder already in place.
\Loading the data...


In [26]:
dfs['premier-league'].head(10)

,Timestamp,Home team,Away team,Goal (home),Goal (away),Date
0,18.03. 21:00,Wolves,Leeds,2,3,2022-03-18 21:00:00
1,17.03. 20:45,Everton,Newcastle,1,0,2022-03-17 20:45:00
2,16.03. 21:15,Arsenal,Liverpool,0,2,2022-03-16 21:15:00
3,16.03. 20:30,Brighton,Tottenham,0,2,2022-03-16 20:30:00
4,14.03. 21:00,Crystal Palace,Manchester City,0,0,2022-03-14 21:00:00
5,13.03. 17:30,Arsenal,Leicester,2,0,2022-03-13 17:30:00
6,13.03. 15:00,Chelsea,Newcastle,1,0,2022-03-13 15:00:00
7,13.03. 15:00,Everton,Wolves,0,1,2022-03-13 15:00:00
8,13.03. 15:00,Leeds,Norwich,2,1,2022-03-13 15:00:00
9,13.03. 15:00,Southampton,Watford,1,2,2022-03-13 15:00:00


In [23]:
dfs['premier-league'].tail(10)

,Timestamp,Home team,Away team,Goal (home),Goal (away),Date
275,15.08. 17:30,Tottenham,Manchester City,1,0,2021-08-15 17:30:00
276,15.08. 15:00,Newcastle,West Ham,2,4,2021-08-15 15:00:00
277,14.08. 18:30,Norwich,Liverpool,0,3,2021-08-14 18:30:00
278,14.08. 16:00,Burnley,Brighton,1,2,2021-08-14 16:00:00
279,14.08. 16:00,Chelsea,Crystal Palace,3,0,2021-08-14 16:00:00
280,14.08. 16:00,Everton,Southampton,3,1,2021-08-14 16:00:00
281,14.08. 16:00,Leicester,Wolves,1,0,2021-08-14 16:00:00
282,14.08. 16:00,Watford,Aston Villa,3,2,2021-08-14 16:00:00
283,14.08. 13:30,Manchester Utd,Leeds,5,1,2021-08-14 13:30:00
284,13.08. 21:00,Brentford,Arsenal,2,0,2021-08-13 21:00:00


In [28]:
leagueOfInterest = dfs['premier-league'].copy(deep=True)

## Splitting the data into two parts, for prediction and base data, to calculate the expected goal.
# testData = leagueOfInterest[leagueOfInterest.Date>=str(leagueOfInterest.Date.max().date())]
# trainData = leagueOfInterest[leagueOfInterest.Date<str(leagueOfInterest.Date.max().date())]
dateOfInterest = '2022-03-16'
testData = leagueOfInterest[leagueOfInterest.Date>=dateOfInterest]
trainData = leagueOfInterest[leagueOfInterest.Date<dateOfInterest]

# Getting the teams
teams = leagueOfInterest.loc[:,'Home team'].unique()

avgGoalsHomeList = []
avgGoalsAwayList = []
avgLastFiveList = []
lastFiveList = []
lastFiveHomeList = []
lastFiveAwayList = []

for i,team in enumerate(teams):

    teamStats = trainData[(trainData.loc[:,'Home team']==team) | (trainData.loc[:,'Away team']==team)]
    homeGames = trainData[trainData.loc[:,'Home team']==team]
    awayGames = trainData[trainData.loc[:,'Away team']==team]

    lastFive = teamStats.head(5)
    lastFiveHome = homeGames.head(5)
    lastFiveAway = awayGames.head(5)

    goalsHome = homeGames.loc[:,'Goal (home)'].astype(int)
    goalsAway = awayGames.loc[:,'Goal (away)'].astype(int)

    avgGoalsHome = goalsHome.sum()/min(goalsHome.shape[0],5)
    avgGoalsAway = goalsAway.sum()/min(goalsAway.shape[0],5)

    allGoals = pd.concat([goalsHome,goalsAway])
    sortedIndex = allGoals.index.sort_values()

    avgLastFive = allGoals[sortedIndex][0:5].sum()/min(allGoals.shape[0],5)

    avgGoalsHomeList.append(avgGoalsHome)
    avgGoalsAwayList.append(avgGoalsAway)
    avgLastFiveList.append(avgLastFive)
    lastFiveList.append(lastFive)
    lastFiveHomeList.append(lastFiveHome)
    lastFiveAwayList.append(lastFiveAway)

overview = pd.DataFrame({'Team':teams,
                            'avgGoalsHome':avgGoalsHomeList,
                            'avgGoalsAway':avgGoalsAwayList,
                            'avgLastFive':avgLastFiveList
                            }
                        ).sort_values('Team').reset_index(drop=True).round(2)

# overviewGames = pd.DataFrame({'Team':teams,
#                                 'LastFive':lastFiveList,
#                                 'LastFiveHome':lastFiveHomeList,
#                                 'LastFiveAway':lastFiveAwayList
#                             }
#                         ).sort_values('Team').reset_index(drop=True)

print(overview,end='\n'*2)

               Team  avgGoalsHome  avgGoalsAway  avgLastFive
0           Arsenal           4.8           3.8          2.0
1       Aston Villa           4.8           3.4          2.0
2         Brentford           3.2           3.2          1.2
3          Brighton           2.0           3.2          0.2
4           Burnley           2.0           2.4          0.4
5           Chelsea           5.6           5.8          2.2
6    Crystal Palace           4.4           3.4          1.4
7           Everton           3.6           2.0          0.6
8             Leeds           3.4           2.8          0.4
9         Leicester           4.4           3.6          1.2
10        Liverpool           7.4           7.2          2.6
11  Manchester City           8.0           5.6          2.2
12   Manchester Utd           4.8           4.8          2.0
13        Newcastle           4.0           2.4          1.4
14          Norwich           2.0           1.6          0.8
15      Southampton     

## Calculating the probabilty of both teams to score


In [29]:
testData

,Timestamp,Home team,Away team,Goal (home),Goal (away),Date
0,18.03. 21:00,Wolves,Leeds,2,3,2022-03-18 21:00:00
1,17.03. 20:45,Everton,Newcastle,1,0,2022-03-17 20:45:00
2,16.03. 21:15,Arsenal,Liverpool,0,2,2022-03-16 21:15:00
3,16.03. 20:30,Brighton,Tottenham,0,2,2022-03-16 20:30:00


In [30]:
## Calculatiing for all matches to be played

for i in np.arange(testData.shape[0]):
    
    avgGoalHome = overview[overview.Team == testData.loc[i,'Home team']].avgGoalsHome
    avgGoalAway = overview[overview.Team == testData.loc[i,'Away team']].avgGoalsAway
    avgLastFive1 = overview[overview.Team == testData.loc[i,'Home team']].avgLastFive
    avgLastFive2 = overview[overview.Team == testData.loc[i,'Away team']].avgLastFive
    
    
    testData.loc[i,'ProbabilityHomeAway'] = calculateProb(avgGoalHome,avgGoalAway)
    testData.loc[i,'ProbabilityLastFive'] = calculateProb(avgLastFive1,avgLastFive2)

C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [31]:
testData.sort_values('ProbabilityHomeAway',ascending=False)

,Timestamp,Home team,Away team,Goal (home),Goal (away),Date,ProbabilityHomeAway,ProbabilityLastFive
2,16.03. 21:15,Arsenal,Liverpool,0,2,2022-03-16 21:15:00,0.9911,0.8004
1,17.03. 20:45,Everton,Newcastle,1,0,2022-03-17 20:45:00,0.8844,0.3399
0,18.03. 21:00,Wolves,Leeds,2,3,2022-03-18 21:00:00,0.8821,0.2304
3,16.03. 20:30,Brighton,Tottenham,0,2,2022-03-16 20:30:00,0.8453,0.1702


In [160]:
testData.sort_values('ProbabilityLastFive',ascending=False)

,Timestamp,Home team,Away team,Goal (home),Goal (away),Date,ProbabilityHomeAway,ProbabilityLastFive
4,19.02. 16:00,Crystal Palace,Chelsea,0,1,2022-02-19 16:00:00,0.5621,0.6013
6,19.02. 16:00,Southampton,Everton,2,0,2022-02-19 16:00:00,0.3996,0.6013
0,19.02. 18:30,Manchester City,Tottenham,2,3,2022-02-19 18:30:00,0.4720,0.4596
5,19.02. 16:00,Liverpool,Norwich,3,1,2022-02-19 16:00:00,0.0000,0.4012
3,19.02. 16:00,Brighton,Burnley,0,3,2022-02-19 16:00:00,0.2487,0.3766
2,19.02. 16:00,Aston Villa,Watford,0,1,2022-02-19 16:00:00,0.5466,0.2964
1,19.02. 16:00,Arsenal,Brentford,2,1,2022-02-19 16:00:00,0.3194,0.2932


In [88]:
## Example

# Expected goals by the the two oppsing teams.
expg1 = 1.1
expg2 = 1.9
# The upper limit of how many goals to compute probabilities for.
maxgoal = 15

## Lists for the individual probabilities
pb1 = []
pb2 = []

for i in np.arange(maxgoal):
    pb1.append(poisson.pmf(i, expg1))
    pb2.append(poisson.pmf(i, expg2))

# Assuming the independent Poisson model.
probmat = np.matmul(np.array(pb1).reshape((15,1)),
                    np.transpose(np.array(pb2).reshape((15,1))))
    
# Compute the BTTS probability using the formula from the paper.
prob_btts = 1 - (sum(probmat[1:probmat.shape[0],0]) + \
                 sum(probmat[0,1:probmat.shape[1]]) + \
                 probmat[0,0]) 

In [89]:
prob_btts

0.5673473661025176

In [136]:
calculateProb(1.1,1.9)

0.5673